In [1]:
import requests
import json

TRAPI Documentation: https://github.com/NCATSTranslator/ReasonerAPI

Most TRAPI documents contain a `message` key.  Within that `message` are a `query_graph` denoting the user query,
a `knowledge_graph` consisting of the union of all nodes and edges that match the `query_graph` pattern, and a list of `results` that bind `query_graph` elements to `knowledge_graph` elements.

The following message contains only a `query_graph`.  This query graph consists of 3 nodes connected together in a line.   Two of the nodes (`n00` and `n02`) have specified identifiers, while the middle node of the line does not.  Rather the middle node has a list of `categories` that are acceptable.

This query asks "Find me a Biological Process or Activity, or a Gene, or a Pathway that is related to both `PUBCHEM.COMPOUND:644073` (Buprenorphine) and `HP:0000217` (Drymouth).

In [2]:
query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:entity_nega..."]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ["PUBCHEM.COMPOUND:644073"],
            "categories": ["biolink:ChemicalEntity"]
          },
          "n01": {
              "categories": ["biolink:BiologicalProcessOrActivity","biolink:Gene","biolink:Pathway"]
          },
          "n02": {
            "ids": ["HP:0000217"],
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


This query can be sent to various components of Translator as needed.  The most comprehensive way to query translator is to post this query to the Automated Relay System (ARS).

In [3]:
ars_submit_url = "https://ars-prod.transltr.io/ars/api/submit"
response = requests.post(ars_submit_url,json=query)

In [4]:
print(response.status_code)

201


The ARS returned a 201 success code.  It also returned as part of the respone payload a key that we can use to monitor progress and look up results.

In [5]:
PK = response.json()['pk']
print(PK)

99ca200b-6f74-4173-b874-2b1e5ed09a37


The ARAX UI provides a convenient way to view results by PK:

In [6]:
arax_url = f'https://arax.ncats.io/?source=ARS&id={PK}'
print(arax_url)

https://arax.ncats.io/?source=ARS&id=99ca200b-6f74-4173-b874-2b1e5ed09a37


You will have noticed that the specified nodes `n00` and `n02` are defined in terms of identifiers, not names.  All nodes and edges in Translator queries and results are defined by CURIES (Compact URIs).  There are two tools in Translator to help navigate identifiers and queries.   

The name-resolver: https://name-resolution-sri.renci.org/docs has a lookup function that can take a string and return potential identifiers.  Here, we look up the string "alzheimer"

In [16]:
results = requests.post('https://name-resolution-sri.renci.org/lookup?string=alzheimer&offset=0&limit=10')

In [18]:
print(json.dumps(results.json(),indent=4))

{
    "MONDO:0004975": [
        "Alzheimer",
        "Alzheimers",
        "Alzheimer's",
        "ALZHEIMER DIS",
        "ALZHEIMERS DIS",
        "Alzheimer Disease",
        "ALZHEIMER DISEASE",
        "Alzheimer disease",
        "Alzheimer Dementia",
        "Alzheimers disease",
        "DEMENTIA ALZHEIMER",
        "Alzheimer syndrome",
        "alzheimers disease",
        "Disease, Alzheimer",
        "Alzheimer Syndrome",
        "alzheimer diseases",
        "Disease;Alzheimers",
        "Alzheimers Disease",
        "Alzheimer dementia",
        "Alzheimer's Disease",
        "Alzheimer's disease",
        "Alzheimers Dementia",
        "Alzheimers dementia",
        "ALZHEIMER'S DISEASE",
        "alzheimer's disease",
        "Alzheimer Sclerosis",
        "Alzheimer Dementias",
        "Alzheimer sclerosis",
        "Dementia, Alzheimer",
        "alzheimers dementia",
        "dementia alzheimers",
        "sclerosis; Alzheimer",
        "Disease, Alzheimer's",
     

The node normalizer (https://nodenormalization-sri.renci.org/docs) takes CURIES and returns all other CURIES that are synonymous with the input. It also returns labels for the node, the biolink classes of the node, and often the information content of the node.

In [21]:
nn_query = {
  "curies": [
    "MONDO:0004975",
  ],
  "conflate": True
}
results = requests.post('https://nodenormalization-sri.renci.org/get_normalized_nodes',json=nn_query)

In [22]:
print(json.dumps(results.json(),indent=4))

{
    "MONDO:0004975": {
        "id": {
            "identifier": "MONDO:0004975",
            "label": "Alzheimer disease"
        },
        "equivalent_identifiers": [
            {
                "identifier": "MONDO:0004975",
                "label": "Alzheimer disease"
            },
            {
                "identifier": "DOID:10652",
                "label": "Alzheimer's disease"
            },
            {
                "identifier": "EFO:0000249",
                "label": "Alzheimer's disease"
            },
            {
                "identifier": "UMLS:C0002395",
                "label": "Alzheimer's Disease"
            },
            {
                "identifier": "UMLS:C0011265",
                "label": "Presenile dementia"
            },
            {
                "identifier": "UMLS:C0276496",
                "label": "Familial Alzheimer Disease (FAD)"
            },
            {
                "identifier": "UMLS:C0494463",
                "label":